In [3]:
import random
#https://github.com/BaijayantaRoy/Medium-Article/blob/master/A_Star.ipynb
#https://github.com/nas-programmer/path-finding/blob/master/astar.py
import pygame
import sys
import math
import time
pygame.init()

# Screen Properties
WIDTH = 800
HEIGHT = 800
SCREEN_SIZE = (WIDTH, HEIGHT)
WINDOW = pygame.display.set_mode(SCREEN_SIZE)
FPS = 60
Clock = pygame.time.Clock()
TOTAL_COL = 40
TOTAL_ROW = 40
CELL_WIDTH = WIDTH // TOTAL_COL
CELL_HEIGHT = HEIGHT // TOTAL_ROW
pygame.display.set_caption("Algorithm Vis")

BLACK = (0, 0, 0)
DARK_BLUE = (200, 200, 200)
GREEN = (0, 255, 25)
LIME = (0, 200, 95)
RED = (255, 0, 0)
PINK = (255, 0, 205)
CYAN = (0, 255, 255)
YELLOW = (255, 255, 0)

Grid = []
path = []
queue = []
openSet, closeSet = [], []
cost = {}

class Node:
    def __init__(self, row, col):
        self.x = row
        self.y = col
        self.neighbour = []
        self.parentNode = None
        self.rect = pygame.Rect(CELL_WIDTH * self.x, CELL_HEIGHT * self.y, CELL_WIDTH - 1, CELL_HEIGHT - 1)
        self.open = False
        self.closed = False
        self.block = False
        self.f = 0
        self.g = 0
        self.h = 0
    def add_neighbors(self, grid):
        if self.x > 0:
            self.neighbour.append(grid[self.x - 1][self.y])
        if self.x < TOTAL_COL - 1:
            self.neighbour.append(grid[self.x + 1][self.y])
        if self.y > 0:
            self.neighbour.append(grid[self.x][self.y - 1])
        if self.y < TOTAL_ROW - 1:
            self.neighbour.append(grid[self.x][self.y + 1])
        if self.x < TOTAL_COL - 1 and self.y < TOTAL_ROW - 1:
            self.neighbour.append(grid[self.x + 1][self.y + 1])
        if self.x < TOTAL_COL - 1 and self.y > 0:
            self.neighbour.append(grid[self.x + 1][self.y - 1])
        if self.x > 0 and self.y < TOTAL_ROW - 1:
            self.neighbour.append(grid[self.x - 1][self.y + 1])
        if self.x > 0 and self.y > 0:
            self.neighbour.append(grid[self.x - 1][self.y - 1])

    def DrawNode(self):
        pygame.draw.rect(WINDOW, DARK_BLUE, self.rect)

    def DrawClosed(self):
        pygame.draw.rect(WINDOW, YELLOW, self.rect)

    def DrawBlock(self):
        pygame.draw.rect(WINDOW, BLACK, self.rect)

    def DrawOpen(self):
        pygame.draw.rect(WINDOW, LIME, self.rect)

    def DrawPath(self):
        pygame.draw.rect(WINDOW, CYAN, self.rect)

    def DrawSource(self):
        pygame.draw.rect(WINDOW, PINK, self.rect)
    def DrawDestination(self):
        pygame.draw.rect(WINDOW, RED, self.rect)

def create_grid():
    for i in range(TOTAL_ROW):
        arr = []
        for j in range(TOTAL_COL):
            node = Node(i, j)
            cost[node] = 10000
            arr.append(node)
        Grid.append(arr)

def heuristics(a, b):
    dstX = abs(a.x-b.x);
    dstY = abs(a.y-b.y);
    
    if(dstX>dstY):
        return 14*dstY + 10*(dstX-dstY)
    else:
        return 14*dstX + 10*(dstY-dstX)
    #return math.sqrt((a.x - b.x)**2 + abs(a.y - b.y)**2)

def add_neighbor():
    for i in range(TOTAL_ROW):
        for j in range(TOTAL_COL):
            Grid[i][j].add_neighbors(Grid)

def close():
    print("Closing")
    pygame.display.quit()
    pygame.quit()
    sys.exit()

def RetracePath(startNode, endNode):
    currentNode = endNode
    
    while currentNode != startNode:
        path.append(currentNode)
        currentNode = currentNode.parent
    
    
def show_update(source, destination):
    WINDOW.fill(BLACK)
    for i in range(TOTAL_ROW):
        for j in range(TOTAL_COL):
            node = Grid[i][j]
            node.DrawNode()
            if node in path:
                node.DrawPath()
            elif node.closed:
                node.DrawClosed()
            elif node.open:
                node.DrawOpen()
            if node.block:
                node.DrawBlock()
            if node == source:
                node.DrawSource()
            if node == destination:
                node.DrawDestination()
            pass
    pygame.display.flip()

def main():
    WINDOW.fill((0,0,0))
    create_grid()
    add_neighbor()
    isDrawingWall = True
    destination = Grid[18][11]
    source = Grid[30][13]
    isPathFound = False
    show_update(source,destination)
    counter=0;
    max_count=(len(Grid) // 2) ** 10
    currentHoverNode = Grid[0][0]
    openSet.append(source)
    source.open = True
    currentNode = openSet[0]
    
    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                close()
            if event.type == pygame.MOUSEBUTTONDOWN:
                if pygame.mouse.get_pressed(3):
                    if isDrawingWall:
                        col = event.pos[0] // CELL_WIDTH
                        row = event.pos[1] // CELL_HEIGHT
                        if currentHoverNode != Grid[col][row]:
                            currentHoverNode = Grid[col][row]
                            if currentHoverNode!=source and currentHoverNode!=destination :
                                currentHoverNode.block = not currentHoverNode.block
                                show_update(source,destination)
            if event.type == pygame.MOUSEMOTION:
                if pygame.mouse.get_pressed()[0]:
                    if isDrawingWall:
                        col = event.pos[0] // CELL_WIDTH
                        row = event.pos[1] // CELL_HEIGHT
                        if currentHoverNode != Grid[col][row]:
                            currentHoverNode = Grid[col][row]
                            if currentHoverNode!=source and currentHoverNode!=destination :
                                currentHoverNode.block = not currentHoverNode.block
                                show_update(source,destination)
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_p:
                    if len(openSet) > 0 and not isPathFound:
                        currentNode = openSet[0]
                        print("....................")
                        for node in openSet:
                            print("G",node.g,"H",node.h,"F",node.f)
                            if node.f<currentNode.f or (node.f == currentNode.f and node.h < currentNode.h):
                                currentNode = node
                        print("Selected: G ",currentNode.g,"H",currentNode.h,"F",currentNode.f)
                        print("....................")
                        openSet.remove(currentNode)
                        currentNode.closed = True
                        
                        if currentNode == destination:
                            isPathFound=True
                            RetracePath(source,destination)
                            break
                        for neighbourNode in currentNode.neighbour:
                            if neighbourNode.block or neighbourNode.closed:
                                continue

                            newMoveCostToNeighbour = currentNode.g + heuristics(currentNode, neighbourNode)
                            
                            
                            if newMoveCostToNeighbour < neighbourNode.g or not neighbourNode.open:
                                
                                neighbourNode.g = newMoveCostToNeighbour
                                neighbourNode.h = heuristics(neighbourNode, destination)
                                neighbourNode.f = neighbourNode.g + neighbourNode.h
                                neighbourNode.parent = currentNode
                                
                                if not neighbourNode.open:
                                    openSet.append(neighbourNode)
                                    neighbourNode.open = True
                    show_update(source,destination)

if __name__ == "__main__":
    main()

....................
G 0 H 0 F 0
Selected: G  0 H 0 F 0
....................
....................
G 10 H 118 F 128
G 10 H 138 F 148
G 10 H 124 F 134
G 10 H 132 F 142
G 14 H 142 F 156
G 14 H 134 F 148
G 14 H 122 F 136
G 14 H 114 F 128
Selected: G  14 H 114 F 128
....................
....................
G 10 H 118 F 128
G 10 H 138 F 148
G 10 H 124 F 134
G 10 H 132 F 142
G 14 H 142 F 156
G 14 H 134 F 148
G 14 H 122 F 136
G 24 H 104 F 128
G 24 H 110 F 134
G 28 H 120 F 148
G 28 H 108 F 136
G 28 H 100 F 128
Selected: G  28 H 100 F 128
....................
....................
G 10 H 118 F 128
G 10 H 138 F 148
G 10 H 124 F 134
G 10 H 132 F 142
G 14 H 142 F 156
G 14 H 134 F 148
G 14 H 122 F 136
G 24 H 104 F 128
G 24 H 110 F 134
G 28 H 120 F 148
G 28 H 108 F 136
G 38 H 90 F 128
G 38 H 104 F 142
G 42 H 114 F 156
G 42 H 94 F 136
G 42 H 94 F 136
Selected: G  38 H 90 F 128
....................
....................
G 10 H 118 F 128
G 10 H 138 F 148
G 10 H 124 F 134
G 10 H 132 F 142
G 14 H 142 F 156

....................
G 10 H 138 F 148
G 10 H 124 F 134
G 10 H 132 F 142
G 14 H 142 F 156
G 14 H 134 F 148
G 14 H 122 F 136
G 28 H 120 F 148
G 38 H 104 F 142
G 34 H 114 F 148
G 42 H 94 F 136
G 52 H 84 F 136
G 62 H 74 F 136
G 72 H 64 F 136
G 82 H 54 F 136
G 82 H 54 F 136
G 92 H 44 F 136
G 92 H 44 F 136
G 102 H 34 F 136
G 102 H 34 F 136
G 24 H 112 F 136
G 34 H 102 F 136
G 44 H 92 F 136
G 54 H 82 F 136
G 38 H 124 F 162
Selected: G  10 H 124 F 134
....................
....................
G 10 H 138 F 148
G 10 H 132 F 142
G 14 H 142 F 156
G 14 H 134 F 148
G 14 H 122 F 136
G 20 H 120 F 140
G 38 H 104 F 142
G 34 H 114 F 148
G 42 H 94 F 136
G 52 H 84 F 136
G 62 H 74 F 136
G 72 H 64 F 136
G 82 H 54 F 136
G 82 H 54 F 136
G 92 H 44 F 136
G 92 H 44 F 136
G 102 H 34 F 136
G 102 H 34 F 136
G 24 H 112 F 136
G 34 H 102 F 136
G 44 H 92 F 136
G 54 H 82 F 136
G 38 H 124 F 162
G 24 H 130 F 154
Selected: G  102 H 34 F 136
....................
....................
G 10 H 138 F 148
G 10 H 132 F 142
G 14 H 14

....................
G 10 H 138 F 148
G 10 H 132 F 142
G 14 H 142 F 156
G 14 H 134 F 148
G 38 H 104 F 142
G 34 H 114 F 148
G 30 H 124 F 154
G 24 H 130 F 154
G 106 H 38 F 144
G 96 H 48 F 144
G 106 H 38 F 144
G 96 H 48 F 144
G 92 H 58 F 150
G 86 H 58 F 144
G 76 H 68 F 144
G 66 H 78 F 144
G 56 H 88 F 144
G 58 H 86 F 144
G 48 H 96 F 144
G 52 H 98 F 150
G 38 H 106 F 144
G 56 H 108 F 164
G 28 H 116 F 144
G 24 H 126 F 150
G 28 H 136 F 164
G 34 H 134 F 168
Selected: G  38 H 104 F 142
....................
....................
G 10 H 138 F 148
G 10 H 132 F 142
G 14 H 142 F 156
G 14 H 134 F 148
G 34 H 114 F 148
G 30 H 124 F 154
G 24 H 130 F 154
G 106 H 38 F 144
G 96 H 48 F 144
G 106 H 38 F 144
G 96 H 48 F 144
G 92 H 58 F 150
G 86 H 58 F 144
G 76 H 68 F 144
G 66 H 78 F 144
G 56 H 88 F 144
G 58 H 86 F 144
G 48 H 96 F 144
G 52 H 98 F 150
G 38 H 106 F 144
G 48 H 108 F 156
G 28 H 116 F 144
G 24 H 126 F 150
G 28 H 136 F 164
G 34 H 134 F 168
G 52 H 118 F 170
Selected: G  10 H 132 F 142
.................

....................
G 10 H 138 F 148
G 14 H 142 F 156
G 14 H 134 F 148
G 30 H 124 F 154
G 24 H 130 F 154
G 92 H 58 F 150
G 52 H 98 F 150
G 48 H 108 F 156
G 24 H 126 F 150
G 20 H 136 F 156
G 34 H 134 F 168
G 44 H 118 F 162
G 24 H 146 F 170
G 110 H 42 F 152
G 106 H 52 F 158
G 110 H 42 F 152
G 100 H 52 F 152
G 120 H 38 F 158
G 110 H 62 F 172
G 90 H 62 F 152
G 80 H 72 F 152
G 70 H 82 F 152
G 66 H 92 F 158
G 62 H 90 F 152
G 52 H 100 F 152
G 70 H 102 F 172
G 42 H 110 F 152
G 38 H 120 F 158
G 42 H 130 F 172
G 48 H 128 F 176
Selected: G  14 H 134 F 148
....................
....................
G 10 H 138 F 148
G 14 H 142 F 156
G 30 H 124 F 154
G 24 H 130 F 154
G 92 H 58 F 150
G 52 H 98 F 150
G 48 H 108 F 156
G 24 H 126 F 150
G 20 H 136 F 156
G 34 H 134 F 168
G 44 H 118 F 162
G 24 H 146 F 170
G 110 H 42 F 152
G 106 H 52 F 158
G 110 H 42 F 152
G 100 H 52 F 152
G 120 H 38 F 158
G 110 H 62 F 172
G 90 H 62 F 152
G 80 H 72 F 152
G 70 H 82 F 152
G 66 H 92 F 158
G 62 H 90 F 152
G 52 H 100 F 152
G 70 

....................
G 14 H 142 F 156
G 30 H 124 F 154
G 24 H 130 F 154
G 48 H 108 F 156
G 20 H 136 F 156
G 34 H 134 F 168
G 44 H 118 F 162
G 24 H 146 F 170
G 106 H 52 F 158
G 120 H 38 F 158
G 102 H 62 F 164
G 66 H 92 F 158
G 62 H 102 F 164
G 38 H 120 F 158
G 34 H 130 F 164
G 48 H 128 F 176
G 24 H 144 F 168
G 20 H 148 F 168
G 28 H 140 F 168
G 24 H 152 F 176
G 66 H 112 F 178
G 38 H 140 F 178
G 120 H 52 F 172
G 124 H 56 F 180
G 124 H 48 F 172
G 114 H 52 F 166
G 104 H 56 F 160
G 124 H 48 F 172
G 94 H 66 F 160
G 84 H 76 F 160
G 80 H 86 F 166
G 84 H 96 F 180
G 66 H 94 F 160
G 56 H 104 F 160
G 76 H 84 F 160
G 52 H 114 F 166
G 56 H 124 F 180
Selected: G  30 H 124 F 154
....................
....................
G 14 H 142 F 156
G 24 H 130 F 154
G 48 H 108 F 156
G 20 H 136 F 156
G 34 H 134 F 168
G 44 H 118 F 162
G 24 H 146 F 170
G 106 H 52 F 158
G 120 H 38 F 158
G 102 H 62 F 164
G 66 H 92 F 158
G 62 H 102 F 164
G 38 H 120 F 158
G 34 H 130 F 164
G 40 H 128 F 168
G 24 H 144 F 168
G 20 H 148 F 168

SystemExit: 

In [2]:
Grid[0][0] == Grid[0][1]

False

In [8]:
import random
#https://github.com/BaijayantaRoy/Medium-Article/blob/master/A_Star.ipynb
#https://github.com/nas-programmer/path-finding/blob/master/astar.py
import pygame
import sys
import math
import time
pygame.init()

# Screen Properties
WIDTH = 800
HEIGHT = 800
SCREEN_SIZE = (WIDTH, HEIGHT)
WINDOW = pygame.display.set_mode(SCREEN_SIZE)
FPS = 60
Clock = pygame.time.Clock()
TOTAL_COL = 40
TOTAL_ROW = 40
CELL_WIDTH = WIDTH // TOTAL_COL
CELL_HEIGHT = HEIGHT // TOTAL_ROW
pygame.display.set_caption("Algorithm Vis")

BLACK = (0, 0, 0)
DARK_BLUE = (200, 200, 200)
GREEN = (0, 255, 25)
LIME = (0, 200, 95)
RED = (255, 0, 0)
PINK = (255, 0, 205)
CYAN = (0, 255, 255)
YELLOW = (255, 255, 0)

Grid = []
path = []
queue = []
openSet, closeSet = [], []
cost = {}

class Node:
    def __init__(self, row, col):
        self.x = row
        self.y = col
        self.neighbour = []
        self.parentNode = None
        self.rect = pygame.Rect(CELL_WIDTH * self.x, CELL_HEIGHT * self.y, CELL_WIDTH - 1, CELL_HEIGHT - 1)
        self.open = False
        self.closed = False
        self.block = False
        self.f = 0
        self.g = 0
        self.h = 0
    def add_neighbors(self, grid):
        if self.x > 0:
            self.neighbour.append(grid[self.x - 1][self.y])
        if self.x < TOTAL_COL - 1:
            self.neighbour.append(grid[self.x + 1][self.y])
        if self.y > 0:
            self.neighbour.append(grid[self.x][self.y - 1])
        if self.y < TOTAL_ROW - 1:
            self.neighbour.append(grid[self.x][self.y + 1])
        if self.x < TOTAL_COL - 1 and self.y < TOTAL_ROW - 1:
            self.neighbour.append(grid[self.x + 1][self.y + 1])
        if self.x < TOTAL_COL - 1 and self.y > 0:
            self.neighbour.append(grid[self.x + 1][self.y - 1])
        if self.x > 0 and self.y < TOTAL_ROW - 1:
            self.neighbour.append(grid[self.x - 1][self.y + 1])
        if self.x > 0 and self.y > 0:
            self.neighbour.append(grid[self.x - 1][self.y - 1])

    def DrawNode(self):
        pygame.draw.rect(WINDOW, DARK_BLUE, self.rect)

    def DrawClosed(self):
        pygame.draw.rect(WINDOW, YELLOW, self.rect)

    def DrawBlock(self):
        pygame.draw.rect(WINDOW, BLACK, self.rect)

    def DrawOpen(self):
        pygame.draw.rect(WINDOW, LIME, self.rect)

    def DrawPath(self):
        pygame.draw.rect(WINDOW, CYAN, self.rect)

    def DrawSource(self):
        pygame.draw.rect(WINDOW, PINK, self.rect)
    def DrawDestination(self):
        pygame.draw.rect(WINDOW, RED, self.rect)

def create_grid():
    for i in range(TOTAL_ROW):
        arr = []
        for j in range(TOTAL_COL):
            node = Node(i, j)
            cost[node] = 10000
            arr.append(node)
        Grid.append(arr)

def heuristics(a, b):
    dstX = abs(a.x-b.x);
    dstY = abs(a.y-b.y);
    if(dstX>dstY):
        return 14*dstY + 10*(dstX-dstY)
    else:
        return 14*dstX + 10*(dstY-dstX)

def add_neighbor():
    for i in range(TOTAL_ROW):
        for j in range(TOTAL_COL):
            Grid[i][j].add_neighbors(Grid)

def close():
    print("Closing")
    pygame.display.quit()
    pygame.quit()
    sys.exit()

def RetracePath(startNode, endNode):
    currentNode = endNode
    
    while currentNode != startNode:
        path.append(currentNode)
        currentNode = currentNode.parent
    
    
def show_update(source, destination):
    WINDOW.fill(BLACK)
    for i in range(TOTAL_ROW):
        for j in range(TOTAL_COL):
            node = Grid[i][j]
            node.DrawNode()
            if node in path:
                node.DrawPath()
            elif node.closed:
                node.DrawClosed()
            elif node.open:
                node.DrawOpen()
            if node.block:
                node.DrawBlock()
            if node == source:
                node.DrawSource()
            if node == destination:
                node.DrawDestination()
            pass
    pygame.display.flip()

def main():
    WINDOW.fill((0,0,0))
    create_grid()
    add_neighbor()
    isDrawingWall = True
    destination = Grid[30][30]
    source = Grid[0][0]
    isPathFound = False
    show_update(source,destination)
    counter=0;
    max_count=(len(Grid) // 2) ** 10
    currentHoverNode = Grid[0][0]
    openSet.append(source)
    source.open = True
    
    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                close()
            if event.type == pygame.MOUSEBUTTONDOWN:
                if pygame.mouse.get_pressed(3):
                    if isDrawingWall:
                        col = event.pos[0] // CELL_WIDTH
                        row = event.pos[1] // CELL_HEIGHT
                        if currentHoverNode != Grid[col][row]:
                            currentHoverNode = Grid[col][row]
                            if currentHoverNode!=source and currentHoverNode!=destination :
                                currentHoverNode.block = not currentHoverNode.block
                                show_update(source,destination)
            if event.type == pygame.MOUSEMOTION:
                if pygame.mouse.get_pressed()[0]:
                    if isDrawingWall:
                        col = event.pos[0] // CELL_WIDTH
                        row = event.pos[1] // CELL_HEIGHT
                        if currentHoverNode != Grid[col][row]:
                            currentHoverNode = Grid[col][row]
                            if currentHoverNode!=source and currentHoverNode!=destination :
                                currentHoverNode.block = not currentHoverNode.block
                                show_update(source,destination)
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_p:
                    startTime = time.time()
                    while len(openSet) > 0:
                        currentNode = openSet[0]
                        for node in openSet:
                            if node.f<currentNode.f or (node.f == currentNode.f and node.h < currentNode.h):
                                currentNode = node
                        openSet.remove(currentNode)
                        currentNode.closed = True
                        
                        if currentNode == destination:
                            endTime = time.time()
                            isPathFound=True
                            RetracePath(source,destination)
                            openSet.clear()
                            break
                        for neighbourNode in currentNode.neighbour:
                            if neighbourNode.block or neighbourNode.closed:
                                continue

                            newMoveCostToNeighbour = currentNode.g + heuristics(currentNode, neighbourNode)
                            
                            
                            if newMoveCostToNeighbour < neighbourNode.g or not neighbourNode.open:
                                
                                neighbourNode.g = newMoveCostToNeighbour
                                neighbourNode.h = heuristics(neighbourNode, destination)
                                neighbourNode.f = neighbourNode.g + neighbourNode.h
                                neighbourNode.parent = currentNode
                                
                                if not neighbourNode.open:
                                    openSet.append(neighbourNode)
                                    neighbourNode.open = True
                    show_update(source,destination)
                    print("S ", startTime)
                    
                    print(endTime - startTime)
if __name__ == "__main__":
    main()

S  1678424598.3767707
0.0
Closing


SystemExit: 

In [5]:
endTime

NameError: name 'endTime' is not defined